In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Install MMDetection

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.4 MB/s eta 0:14:00tcmalloc: large alloc 1147494400 bytes == 0x39c48000 @  0x7fb112878615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.4 MB/s eta 0:11:25tcmalloc: large alloc 1434370048 bytes == 0x7e29e000 @  0x7fb112878615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.24.1
11.1
GCC 7.3


In [ ]:
from glob import glob
import json
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from tqdm import tqdm

## Perform Inference with An MMDetection Detector

In [ ]:
%ls ../drive/MyDrive/MORZH_TEST

epoch_70.pth  inference.ipynb  test_images/


In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector

# Choose to use a config and initialize the detector
config = 'configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco.py'

# Setup a checkpoint file to load
checkpoint = '../drive/MyDrive/MORZH_TEST/'+'epoch_70.pth'

# Set the device to be used for evaluation
device='cuda:0'

# Load the config
cfg = mmcv.Config.fromfile(config)
# Set pretrained to be None since we do not need pretrained model here
cfg.model.pretrained = None

from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'COCODataset'

cfg.data.test.classes = ('walrus',)

# modify num classes of the model in box head and mask head
cfg.model.roi_head.bbox_head.num_classes = 1
cfg.model.roi_head.mask_head.num_classes = 1

# We can still the pre-trained Mask RCNN model to obtain a higher performance
# cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'
cfg.load_from = '../drive/MyDrive/MORZH_TEST/'+'epoch_70.pth'

# ПОМЕНЯТЬ ТОЛЬКО ТЕСТ
cfg.model.test_cfg.rpn.nms_pre=2000
cfg.model.test_cfg.rpn.max_per_img=1500
cfg.model.test_cfg.rcnn.max_per_img=1000

# cfg.model.test_cfg.rpn.nms.iou_threshold=0.5

# We can initialize the logger for training and have a look
# at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')



# Initialize the detector
model = build_detector(cfg.model)

# Load checkpoint
checkpoint = load_checkpoint(model, checkpoint, map_location=device)

# Set the classes of models for inference
model.CLASSES = checkpoint['meta']['CLASSES']

# We need to set the model's cfg for inference
model.cfg = cfg

# Convert the model to GPU
model.to(device)
# Convert the model into evaluation mode
model.eval()

load checkpoint from local path: ../drive/MyDrive/MORZH_TEST/epoch_70.pth


MaskRCNN(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

From the printed model, we will find that the model does consist of the components that we described earlier. It uses ResNet as its CNN backbone, and has a RPN head and RoI Head. 
The RoI Head includes box head and mask head. In addition, the model has a neural network module, named neck, directly after the CNN backbone. It is a [feature pyramid network (FPN)](https://arxiv.org/abs/1612.03144) for enhancing the multi-scale features.


### Inference with the detector

The model is successfully created and loaded, let's see how good it is. We use the high-level API `inference_detector` implemented in the MMDetection. This API is created to ease the inference process. The details of the codes can be found [here](https://github.com/open-mmlab/mmdetection/blob/master/mmdet/apis/inference.py#L15).

In [ ]:
import cv2
import os

In [ ]:
os.mkdir('test_split_images/')

In [ ]:
xsplit_values = []
files = []

for file in glob('../drive/MyDrive/MORZH_TEST/test_images/*.jpg'):
    
    image = cv2.imread(file).copy()
    
    if image.shape[1] > 4000:
        xsplit = image.shape[1] // 3
        xsplit_values.append(xsplit)
        image1 = image[:, :xsplit, :].copy()
        image2 = image[:, xsplit:2*xsplit, :].copy()
        image3 = image[:, 2*xsplit:, :].copy()
        
        filedir = file.split('/')[-1].split('.')[0]
        os.mkdir(f'test_split_images/{filedir}')
        cv2.imwrite(f'test_split_images/{filedir}/1.jpg', image1)
        cv2.imwrite(f'test_split_images/{filedir}/2.jpg', image2)
        cv2.imwrite(f'test_split_images/{filedir}/3.jpg', image3)
        
    
    elif image.shape[1] > 3000:
        xsplit = image.shape[1] // 2
        xsplit_values.append(xsplit)
        image1 = image[:, :xsplit, :].copy()
        image2 = image[:, xsplit:, :].copy()
        
        filedir = file.split('/')[-1].split('.')[0]
        os.mkdir(f'test_split_images/{filedir}')
        cv2.imwrite(f'test_split_images/{filedir}/1.jpg', image1)
        cv2.imwrite(f'test_split_images/{filedir}/2.jpg', image2)
        
    else:
        xsplit_values.append(0)
        filedir = file.split('/')[-1].split('.')[0]
        os.mkdir(f'test_split_images/{filedir}')
        cv2.imwrite(f'test_split_images/{filedir}/1.jpg', image)

    files.append(file)

In [ ]:
xsplit_df = pd.DataFrame()
xsplit_df['xsplit'] = xsplit_values
xsplit_df['file'] = files
xsplit_df['file'] = xsplit_df['file'].apply(lambda x: x.split('/')[-1].split('.')[0])
xsplit_df.to_csv('xsplit_values.csv', index=False)

In [ ]:
def get_res_df(file):
    result = inference_detector(model, file)
    
    res_coords = []
    for j, mask_res in enumerate(result[0][0]):
        center_x = (mask_res[2] + mask_res[0]) / 2
        center_y = (mask_res[3] + mask_res[1]) / 2
        if mask_res[4] > 0.05:
            res_coords.append([center_x, center_y])
    
    return pd.DataFrame(res_coords, columns=['x', 'y'])

In [ ]:
os.mkdir('../drive/MyDrive/MORZH_TEST/test_results/')

In [ ]:
for filedir in tqdm(glob('test_split_images/*')):
    
    file_csv = filedir.split('/')[-1]

    res_df = pd.DataFrame()
    
    for file in glob(filedir+'/*'):
        
        res_df_temp = get_res_df(file)
        
        xsplit_value = float(file.split('/')[-1].split('.')[0])
        xsplit = xsplit_df[xsplit_df.file == file_csv].xsplit.iloc[0]
        
        res_df_temp['x'] += (xsplit_value-1)*xsplit
        
        res_df = pd.concat([res_df, res_df_temp])
    
    res_df.to_csv(f'../drive/MyDrive/MORZH_TEST/test_results/{file_csv}.csv', index=False)

100%|██████████| 50/50 [00:48<00:00,  1.04it/s]


In [ ]:
os.mkdir('../drive/MyDrive/MORZH_TEST/test_show/')

In [14]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

for file in tqdm(glob('../drive/MyDrive/MORZH_TEST/test_images/*.jpg')):

    show_file = file.split('/')[-1].split('.')[0]

    fig, ax = plt.subplots(figsize=(40, 30))
    im = plt.imread(file)
    ax.imshow(im)

    res_df = pd.read_csv(f'../drive/MyDrive/MORZH_TEST/test_results/{show_file}.csv')
    ax.scatter(x=list(res_df.x.values), y=list(res_df.y.values), c='r', s=30)

    
    fig.savefig(f'../drive/MyDrive/MORZH_TEST/test_show/{show_file}.jpg')

In [ ]:
import os

In [ ]:
os.mkdir('./drive/MyDrive/MORZH_TEST/test_results_post/')

for file in tqdm(glob('./drive/MyDrive/MORZH_TEST/test_results/*.csv')):
    res_df = pd.read_csv(file).copy()
    res_df.x = np.round(res_df.x)
    res_df.y = np.round(res_df.y)
    res_df.to_csv(file.replace('test_results', 'test_results_post'), index=False, sep=',')

100%|██████████| 50/50 [00:00<00:00, 70.30it/s]


'./drive/MyDrive/MORZH_TEST/test_results_post/050.csv'